In [8]:
#import stuff

import csv
import pandas as pd
import numpy as np
import random
from collections import Counter

%store -r races
%store -r races2

In [9]:
#ordered race & nonhispanic/hispanic data based on frequency in general US population -
# generated from ../dp/create_all_usdistribution.csv
real_order = [124, 116, 118, 123, 115, 117, 125, 51, 59, 111, 32, 102, 
              108, 42, 62, 37, 99, 44, 103, 48, 26, 96, 31, 106, 95, 
              45, 105, 38, 41, 107, 119, 97, 33, 43, 30, 94, 13, 79, 
              15, 83, 80, 27, 29, 12, 88, 76, 11, 78, 4, 17, 81, 75, 
              19, 74, 14, 6, 67, 68, 1, 10, 65, 5, 64, 63, 0, 66, 69, 
              71, 70, 2, 73, 8, 7, 84, 3, 72, 91, 77, 85, 20, 82, 21,
              24, 9, 18, 28, 104, 22, 110, 87, 25, 86, 89, 92, 35, 90,
              40, 36, 100, 50, 47, 16, 93, 39, 23, 98, 34, 56, 114, 57, 
              101, 113, 54, 46, 52, 49, 60, 55, 120, 53, 58, 109, 112, 61, 122, 121]
#print(real_order.index(48+63))

In [10]:
############################################################################################################################
#This method imports county data from csv file to pandas df and reformats race data numerically
############################################################################################################################
def import_countydata(county):
    #import csv file as dataframe
    df = pd.read_csv (r'../homemade_data/'+county+'.csv')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    #transfer race representation to include hispanic data:
    #if not hispanic: index is 0-62
    #if hispanic: index is 63-126
        #so, if hispanic, add 63 to the existing race value
    for index, row in df.iterrows():
        d = row['race']
        h = row['hispanic']
        if d in races and h==0:
            df.at[index, 'race'] = real_order.index(races.index(d))
        elif d in races and h==1:
            df.at[index, 'race'] = real_order.index((races.index(d))+63)

    return df

In [11]:
############################################################################################################################
#This method imports state data from csv file to pandas df and reformats race data numerically
#------------------------------
#INPUT:
    #state: state to import and organize data for
#------------------------------
#OUTPUT:
#final_dfs: list containing occurences of each real_order race index
    #ex. final_dfs[race]= rows in state dataset that have the same real order value 
############################################################################################################################
def import_statedata(state):
    #import csv file as dataframe
    df = pd.read_csv (r'state_data/'+state+'.csv')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
   
    for index, row in df.iterrows():        
    #transfer race representation to include hispanic data:
        #if not hispanic: index is 0-62
        #if hispanic: index is 63-126
            #so, if hispanic, add 63 to the existing race value    
        d = row['race']
        h = row['hispanic']
        if d in races2 and h==0:
            df.at[index, 'race'] = real_order.index(races2.index(d))
        elif d in races2 and h==1:
            df.at[index, 'race'] = real_order.index(races2.index(d)+63)

    final_dfs = {}

    #create list of occurence count in df for each real_order race value, sort values inside each by age
    for race_realorder in range(0,126):
        final_dfs[race_realorder] = (df[df['race'] == race_realorder]).sort_values(by='age')

    #returns list of real_orders with corresponding values in dataset  
    return final_dfs


In [12]:
############################################################################################################################
#GET ROWS PRIORITIZED FOR SWAPPING
#Get variables prioritized to be swapped 
#This method returns rows based upon whether they are unique in the dataset
############################################################################################################################

def pick_swapvars(df):
    #all v is collection of all row values
    all_v = []

    #add all rows to list all_v
    for i, row in df.iterrows():
        all_v.append([[row['age'], row['sex'], row['race']], row['id']])
        
    #drop duplicates based upon age, sex, and race from df
    df_drop = df.drop_duplicates(subset = ['age', 'sex', 'race'])
        
    #the list of ids of all rows that contain unique entries
    swap = []
    
    #for each item in uniques, check how many times it occurs in original data
    for i, row in df_drop.iterrows():
        #list containing age, sex, race
        check = [row['age'], row['sex'], row['race']]
        dcount = 0 
        #for all rows in the dataframe
        for a in all_v:
            #check if the age, sex, and races match the row
            if a[0]==check:
                #if they do match, incremement by 1
                dcount +=1
        #if only present once, add to swaps
        if dcount == 1:
            swap.append(row['id']-1)
            
               
    return swap

In [58]:
############################################################################################################################
#GET ALL ROWS TO SWAP
#----------------------------------------------------
#INPUT
    #dframe: full pandas dataset
    #swaprate: % to swap at
    #swap: the list of ids of all rows that contain unique entries
#----------------------------------------------------
#OUTPUT
    #returns list of rows to swap, list of rows to not swap, & # of rows to b swapped
############################################################################################################################

def get_rows_to_swap(dframe, swaprate, swap):
    
    #number of rows we need to swap in the dataset, based upon the given swap rate 
    rows_to_swap = swaprate*dframe.shape[0]
    rows_to_swap = round(rows_to_swap)
    
    #rows to be swapped and not swapped
    swaps = []
    not_swaps = []
    
    #if more rows need to be swapped than there are unique rows present in swap
    if rows_to_swap > len(swap):
        print('here')
        #add all unique rows to swaps
        swaps = swap
        #all rows in dataset to choose from to supplementally swap
        rows = np.arange(0,dframe.shape[0], 1)        
        #find one not already in swaps and add it to swaps
        while len(swaps) < rows_to_swap:
            random_s = random.choice(rows)
            if random_s not in swaps:
                swaps.append(random_s)

    # else if number of rows needed to be swapped is less than or equal to unique rows present in swap
    else:
        #all unique rows in dataset to choose from to swap
        rows2 = np.arange(0,len(swap), 1)
        #find one not already in swaps and add it to swaps 
        while len(swaps) < rows_to_swap:
            random_s = swap[random.choice(rows2)]
            if random_s not in swaps:
                swaps.append(random_s)

    #if not present in swaps, add to not_swaps 
    for i in range(0,dframe.shape[0]):
        if i not in swaps:
            not_swaps.append(i)

    #returns list of rows to swap, list of rows to not swap, & # of rows to b swapped
    return swaps, not_swaps, rows_to_swap

In [59]:
############################################################################################################################
#FIND THE SWAP
#INPUT 
#dframe = dataset containing r 
#r = row to swap
#thresholds for each variable type
#------------------
#OUTPUT
#return row to be swapped, and the thresholds at which this match was made
############################################################################################################################

def find_swap(dframe, r, threshold_age, threshold_sex, threshold_race):
    
    #find similar row in state dataframe
        
    #get data from normal dataframe for r
    age = dframe.at[r, 'age']
    sex = dframe.at[r, 'sex']
    race = dframe.at[r, 'race']
    

    complete = False
    
    #find someone similar in state
    while complete == False:
            
        #filter for race
        new_data = df_state[race]
        i=1            
        while i <=threshold_race:
            if(race-i >= 0):
                new_data = new_data.append(df_state[race-i], ignore_index=True)
            if(race+i < len(real_order)):
                new_data = new_data.append(df_state[race+i], ignore_index=True)
                i+=1
                
        #filter for age
        new_data = new_data[new_data['age']>= int(age-threshold_age)]       
        new_data = new_data[new_data['age']<=int(age+threshold_age)]     

        #filter for sex
        if threshold_sex == 0:
            new_data = new_data[new_data['sex']==sex]

        #check if done (if there are any matches)
        if new_data.shape[0] > 0:
            final = new_data.sample()
            complete = True
            break
                
        #else increment thresholds and try again
        else: 
            threshold_race = threshold_race + 2
            threshold_age = threshold_age + 2

    #return row to be swapped, and the thresholds at which this match was made
    return final, threshold_race, threshold_age

In [60]:
############################################################################################################################
#DO THE SWAP
#INPUT 
#final = row in state dataset to swap
#r = row in block dataset to swap
#df_county = county dataset
#------------------
#OUTPUT
#dframe: new county dataset
############################################################################################################################

def do_swap(dframe, r, final):

    #set new values
    dframe.at[r, 'age'] = final['age']
    dframe.at[r, 'sex'] = final['sex'].values[0]
    dframe.at[r, 'hispanic'] = final['hispanic']
    dframe.at[r, 'race'] = final['race'].values[0]
    dframe.at[r, 'SwapVal'] = True        
    dframe.at[r, 'household_size'] = final['household_size']
    dframe.at[r, 'household_tenure'] = final['household_tenure'].values[0]
    
    return dframe

In [93]:
############################################################################################################################
#SWAPPING IMPLEMENTATION - SIMILARITY BASED --> given a county block dataset and state dataset, swaps at a given 
                                                #swaprate and threshold    
#INPUT:
#dfr = county block dataset 
#swaprate = rate of data to swap out from county block
#swap = list of unique entries prioritized for swapping
#df_state = state dataset
#thresholds: limit to neighboring values we can draw from for similarity
#-----------------------------------
#OUTPUT: 
#dframe = new de-identified county block dataset
############################################################################################################################
def swap(dfr, swaprate, swap, df_state, threshold_age, threshold_sex, threshold_race):
    
    #num_of_swaps: number of swaps remaining to do
    num_of_swaps = 0
    #import dataframe, set all rows to swapval of false
    dframe = dfr.copy()
    dframe['SwapVal'] = 'False'
    
    #get rows to swap, rows to not swap, and number of rows to be swapped
    swaps, not_swaps, rows_to_swap = get_rows_to_swap(dframe, swaprate, swap)

    
    #while still swaps available to make
    while num_of_swaps < rows_to_swap:
        
        #choose a row from swaps to swap, and then remove from swap list
        rows = np.arange(0,len(swaps), 1)
        r = swaps[random.choice(rows)]
        swaps.remove(r)
    
        
#         #get row from state distribution to swap with r, and thresholds that returned this val
#         final, finalt_r, finalt_a = find_swap(dframe, r, threshold_age, threshold_sex, threshold_race)
        
#         #swap row from state distribution with r
#         dframe = do_swap(dframe, r, final)

#         #increment swap count
        num_of_swaps = num_of_swaps +1
     
#     #reset race to their original values 
#     for index, row in dframe.iterrows():
#         #change from real_order indexes to actual race values
#         dframe.at[index, 'race'] = real_order[row['race']]
#         #if hispanic, reset race values to 0-63
#         if dframe.at[index, 'race'] >62:
#             dframe.at[index, 'race'] = int(dframe.at[index, 'race'])-63

    return dframe

In [94]:









################################################### RUNNING THE CODE #####################################################











In [95]:
# state = 'test_state'
# county = 'test_county'


# df = import_countydata(county)
# df_state = import_statedata(state)


In [96]:
# swaprates = [.1]

# threshold_age = 5
# threshold_sex = 1
# threshold_race = 3

# for s in swaprates:
#     print(s)
#     s = float(s)
#     swapped = pick_swapvars(df)           
#     dframe = swap(df, s, swapped, df_state, threshold_age, threshold_sex, threshold_race) 
#     #print(dframe)
#     filename = "swap_runs/"+county+"/similar/swap_"+str(s)+"_a"+str(k)+".csv"
#     csv_orig_data = dframe.to_csv(filename, index = True)
#     k+=1

In [97]:
states = ["pennsylvania", "newmexico", "georgia", "northdakota", "hawaii", "missouri", "massachussets", "vermont"]
counties = ['armstrong', 'cibola', 'fayette', 'grandforks', 'hawaii', 'jefferson', 'nantucket', 'washington']
states = ['test_state']
counties = ['hawaii']

In [109]:
swaprates = np.arange(.43, .46, .01, float)
k=0

threshold_age = 5
threshold_sex = 1
threshold_race = 3


for i in range(0,len(states)):
    county = counties[i]
    state = states[i]
    print(county, state)
    df = import_countydata(county)
    df_state = import_statedata(state)
    swapped = pick_swapvars(df) 
    a = pick_swapvars(df)
    sorteddd = []
    while k <1:
        for s in swaprates:
            print(s)
            s = float(s)
            dframe = swap(df, s, swapped, df_state, threshold_age, threshold_sex, threshold_race) 
            sorteddd.append([s,swapped])
            #print(dframe)
            filename = "swap_runs/"+county+"/similar/swap_"+str(s)+"_"+str(k)+".csv"
            csv_orig_data = dframe.to_csv(filename, index = True)
        k+=1
    #print(all(element == a for element in sorteddd))
    print(a == sorteddd[0][1])
    print(a == sorteddd[1][1])
    print(a == sorteddd[2][1])
    print(a == sorteddd[3][1])

hawaii test_state
0.43
[331, 1244, 1041, 981, 886, 417, 846, 1044, 1120, 938, 992, 1208, 1187, 1168, 966, 905, 1023, 1082, 704, 990, 1219, 1019, 1180, 1056, 123, 1126, 444, 389, 605, 1196, 1224, 893, 419, 328, 945, 491, 1137, 691, 1211, 150, 1145, 1108, 1037, 1231, 175, 39, 941, 56, 1186, 395, 1034, 876, 1178, 1103, 871, 1084, 446, 571, 52, 878, 949, 814, 1118, 935, 883, 849, 539, 851, 946, 344, 1080, 212, 934, 723, 338, 1179, 1153, 865, 974, 1177, 236, 897, 1226, 1043, 1250, 869, 504, 251, 809, 559, 940, 988, 980, 882, 197, 385, 1050, 951, 958, 1009, 568, 78, 731, 794, 325, 920, 1253, 1156, 1241, 289, 1026, 872, 1079, 1131, 1033, 463, 279, 1205, 341, 1242, 1073, 844, 1054, 999, 787, 347, 348, 316, 734, 902, 764, 816, 199, 853, 81, 1174, 1201, 768, 885, 1172, 1092, 825, 1228, 868, 1247, 818, 1243, 977, 1229, 1255, 597, 569, 1240, 550, 744, 342, 409, 486, 1147, 1115, 653, 1039, 416, 1213, 321, 842, 659, 1113, 856, 372, 1124, 1016, 1074, 859, 174, 443, 913, 602, 1035, 1025, 1105, 627, 10

False
False
False
False


In [82]:
r = [1145, 486, 993, 969, 1139, 669, 750, 243, 1107, 344, 829, 1045, 856, 840, 989, 1252, 868, 955, 650, 824, 245, 597, 1213, 872, 578, 1205, 1089, 56, 908, 661, 1191, 825, 311, 954, 627, 1106, 1050, 340, 799, 1031, 373, 1001, 1208, 522, 1115, 1025, 930, 1044, 964, 961, 330, 1126, 762, 1130, 844, 345, 1079, 1241, 978, 729, 1131, 31, 906, 335, 1071, 963, 1202, 1187, 325, 775, 1052, 1088, 236, 1175, 337, 1118, 837, 660, 851, 1054, 974, 751, 735, 882, 839, 718, 1066, 235, 331, 1146, 877, 752, 768, 39, 342, 1007, 1151, 1030, 914, 1043, 1254, 857, 776, 121, 1105, 1059, 1240, 855, 945, 1172, 731, 1143, 848, 1065, 1226, 758, 861, 350, 920, 866, 279, 1039, 1111, 123, 321, 950, 836, 349, 1100, 744, 472, 830, 878, 1035, 466, 320, 333, 846, 1156, 76, 1180, 571, 167, 1246, 1133, 1033, 291, 1211, 52, 322, 694, 663, 68, 1070, 175, 1020, 723, 1249, 200, 1137, 387, 1108, 1006, 1037, 943, 1026, 401, 667, 1153, 1009, 1218, 1074, 683, 220, 980, 777, 1075, 733, 1250, 843, 1160, 1174, 951, 1221, 815, 1104, 1064, 1099, 1245, 609, 372, 835, 1014, 57, 507, 797, 339, 281, 385, 386, 255, 304, 1155, 1073, 191, 1082, 1121, 862, 409, 709, 975, 655, 1096, 1257, 896, 884, 805, 129, 126, 1186, 681, 716, 416, 898, 1011, 707, 936, 774, 926, 1021, 795, 1092, 977, 1225, 847, 287, 1229, 966, 269, 1164, 504, 992, 869, 798, 853, 741, 794, 809, 1222, 949, 985, 933, 712, 932, 860, 305, 881, 976, 327, 1231, 1042, 778, 1142, 947, 35, 1171, 720, 706, 150, 631, 1196, 671, 748, 1102, 289, 1048, 1224, 1236, 1005, 1217, 1080, 13, 1255, 1152, 653, 901, 715, 0, 1199, 161, 903, 842, 294, 491, 119, 704, 879, 1103, 970, 1170, 446, 948, 406, 559, 1198, 780, 1010, 990, 1168, 389, 1122, 1188, 212, 899, 691, 351, 1116, 1227, 136, 833, 117, 814, 355, 417, 885, 343, 145, 939, 1084, 991, 1004, 179, 539, 328, 1195, 1194, 828, 42, 197, 1200, 1210, 821, 1110, 725, 897, 1163, 295, 887, 912, 395, 841, 217, 346, 1190, 1207, 1228, 476, 958, 174, 922, 787, 981, 968, 1251, 443, 1109, 1204, 1047, 316, 893, 444, 701, 770, 1077, 199, 584, 902, 1230, 6, 1056, 973, 1248, 889, 1008, 816, 940, 870, 983, 1091, 1256, 850, 863, 1219, 548, 764, 1081, 736, 1124, 1034, 85, 907, 934, 971, 1179, 550, 849, 1041, 1148, 873, 1193, 588, 1192, 1013, 942, 107, 891, 1233, 140, 498, 251, 1197, 1253, 347, 659, 905, 1049, 533, 693, 925, 81, 1113, 348, 1072, 788, 924, 865, 852, 1206, 732, 859, 569, 1024, 534, 956, 419, 997, 1017, 1147, 598, 883, 568, 965, 1242, 988, 1212, 941, 26, 1244, 913, 1117, 806, 376, 791, 182, 308, 334, 818, 510, 1083, 605, 1189, 1154, 921, 620, 972, 858, 1149, 338, 1141, 1239, 1177, 938, 1120, 662, 404, 871, 886, 144, 501, 890, 1159, 1182, 1057, 673, 1184, 1214, 1032, 481, 1027, 137, 78, 74, 1203, 1012, 332, 674, 656, 831, 329, 264, 1169, 894, 845, 1247, 783, 763, 944, 225, 734, 854, 602, 341, 87, 935, 876, 1123, 967, 909, 1019, 1016, 1093, 110, 875, 1201, 1036, 756, 668, 1178, 1023, 1235, 452]

s = 

print(len(r)-len(s))
print(round(1258/100))
#print(len(t)-len(r))
print(26/1258)

-13
13
0.02066772655007949


In [68]:
print(s)

[0, 6, 13, 26, 31, 35, 39, 42, 52, 56, 57, 68, 74, 76, 78, 81, 85, 87, 107, 110, 117, 119, 121, 122, 123, 126, 129, 136, 137, 140, 144, 145, 150, 161, 167, 174, 175, 179, 182, 191, 197, 199, 200, 212, 217, 220, 225, 235, 236, 238, 243, 245, 251, 255, 264, 269, 279, 281, 287, 289, 291, 294, 295, 304, 305, 308, 311, 316, 320, 321, 322, 325, 327, 328, 329, 330, 331, 332, 333, 334, 335, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 355, 372, 373, 376, 385, 386, 387, 389, 395, 401, 404, 406, 409, 417, 419, 443, 444, 446, 452, 463, 466, 467, 472, 476, 481, 486, 491, 498, 501, 504, 507, 510, 522, 533, 534, 539, 548, 550, 559, 568, 569, 571, 578, 584, 588, 597, 598, 602, 605, 609, 620, 627, 631, 650, 653, 655, 656, 659, 660, 661, 662, 663, 667, 668, 669, 671, 673, 674, 681, 683, 691, 693, 694, 701, 704, 706, 707, 709, 712, 715, 716, 718, 720, 723, 725, 729, 731, 732, 733, 734, 735, 736, 741, 744, 748, 750, 751, 752, 756, 758, 762, 763, 764, 768, 770, 774, 775, 776, 777, 778,